In [259]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# Report
from ydata_profiling import ProfileReport

# Importing the dataset
Firstly, we import the dataset we will use.
Feel free to change the dataset to another tournament

In [260]:
# You can change the file name to any tournament you want to analyze
file_name = "LFL_2023_Summer.csv"
dataset = pd.read_csv("data/" + file_name)

In [261]:
# View the first 10 rows of the dataset
dataset.head(10)

,Unnamed: 0,gameId,patch,participantId,blueTeam,redTeam,role,allInPings,assistMePings,assists,...,wardsKilled,wardsPlaced,win,playerAugment1,playerAugment2,playerAugment3,playerAugment4,playerSubteamId,subteamPlacement,placement
0,0,2816450,13.1,1,LDLC OL,Team GO,TOP,1,13,2,...,3,17,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2816450,13.1,2,LDLC OL,Team GO,JUNGLE,3,7,4,...,12,10,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2816450,13.1,3,LDLC OL,Team GO,MID,3,0,1,...,7,6,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2816450,13.1,4,LDLC OL,Team GO,ADC,1,6,1,...,10,19,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2816450,13.1,5,LDLC OL,Team GO,SUPPORT,2,2,3,...,20,99,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,2816450,13.1,6,LDLC OL,Team GO,TOP,1,4,3,...,17,19,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,2816450,13.1,7,LDLC OL,Team GO,JUNGLE,4,65,9,...,14,12,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,2816450,13.1,8,LDLC OL,Team GO,MID,0,3,7,...,6,5,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,2816450,13.1,9,LDLC OL,Team GO,ADC,0,10,4,...,15,15,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,2816450,13.1,10,LDLC OL,Team GO,SUPPORT,9,66,10,...,12,53,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [262]:
# Showing all the possible attributes

dataset.columns

Index(['Unnamed: 0', 'gameId', 'patch', 'participantId', 'blueTeam', 'redTeam',
       'role', 'allInPings', 'assistMePings', 'assists',
       ...
       'wardsKilled', 'wardsPlaced', 'win', 'playerAugment1', 'playerAugment2',
       'playerAugment3', 'playerAugment4', 'playerSubteamId',
       'subteamPlacement', 'placement'],
      dtype='object', length=133)

# Selecting good attributes
The attributes selected are the ones that are useful for a possible player rating.
The `champExperience` is only available in the V5 of stats API and is not available in the V4.
This is why we make a verification in the dataset if it contains the `champExperience` attribute. Otherwise, we will use the `champLevel` attribute.
```python
if 'champExperience' in dataset.columns:
    useful_columns.append('champExperience')
else:
    useful_columns.append('champLevel')
```

In [263]:
# List of possible useful columns
# This is a prefilter of the columns
useful_columns = ['gameId', 
                  'patch', 
                  'role', 
                  'summonerName',
                  'blueTeam',
                  'redTeam',
                  'teamId',
                  'championName',
                  'kills',
                  'deaths',
                  'assists',
                  'goldEarned',
                  'totalDamageDealtToChampions',
                  'totalMinionsKilled',
                  'totalHeal',
                  'timeCCingOthers',
                  'totalDamageTaken',
                  'damageSelfMitigated',
                  'wardsKilled',
                  'wardsPlaced',
                  'win',
                  ]
# If 'champExperience' is in the dataset, then we can use it
# otherwise, we can use 'champLevel'
if 'champExperience' in dataset.columns:
    useful_columns.append('champExperience')
else:
    useful_columns.append('champLevel')

final_dataset = dataset[useful_columns].copy()

# Let's rename the champLevel to champExperience if it is not in the dataset
if 'champExperience' not in dataset.columns:
    final_dataset.rename(columns={'champLevel': 'champExperience'}, inplace=True)

final_dataset.head(10)

,gameId,patch,role,summonerName,blueTeam,redTeam,teamId,championName,kills,deaths,...,totalDamageDealtToChampions,totalMinionsKilled,totalHeal,timeCCingOthers,totalDamageTaken,damageSelfMitigated,wardsKilled,wardsPlaced,win,champExperience
0,2816450,13.1,TOP,LDLC Kryze,LDLC OL,Team GO,100,KSante,2,3,...,12327,248,8341,35,34985,61638,3,17,False,16686
1,2816450,13.1,JUNGLE,LDLC White,LDLC OL,Team GO,100,Sejuani,1,1,...,6562,32,25100,18,43250,46718,12,10,False,13591
2,2816450,13.1,MID,LDLC Backlund,LDLC OL,Team GO,100,Yone,3,2,...,11888,327,9392,13,27962,31219,7,6,False,20006
3,2816450,13.1,ADC,LDLC Jeskla,LDLC OL,Team GO,100,Xayah,1,3,...,24968,362,8530,14,21575,11561,10,19,False,17569
4,2816450,13.1,SUPPORT,LDLC Zoelys,LDLC OL,Team GO,100,Rakan,0,3,...,4177,42,6329,28,17211,20846,20,99,False,8525
5,2816450,13.1,TOP,GO Ragner,LDLC OL,Team GO,200,Jayce,7,2,...,30804,300,4809,9,21153,21643,17,19,True,20374
6,2816450,13.1,JUNGLE,GO Lyncas,LDLC OL,Team GO,200,Maokai,1,1,...,18938,24,21918,51,28480,43384,14,12,True,14245
7,2816450,13.1,MID,GO toucouille,LDLC OL,Team GO,200,Ahri,2,0,...,33197,318,5756,33,15890,11406,6,5,True,19972
8,2816450,13.1,ADC,GO Jezu,LDLC OL,Team GO,200,Ezreal,2,2,...,26936,321,4528,7,15043,12977,15,15,True,16699
9,2816450,13.1,SUPPORT,GO kamilius,LDLC OL,Team GO,200,Milio,0,2,...,1496,36,7842,10,10837,7710,12,53,True,11877


# Cleaning data
Since some players might have renamed while the tournament (like Targamas who used XL Targamas instead of KC Targamas) we will replace the names with the most used one in the tournament.

In [264]:
from collections import defaultdict

def fix_tags(dataset):
    # Créer un dictionnaire pour stocker les informations des joueurs
    player_info = defaultdict(list)

    # Parcourir le dataset pour collecter les informations des joueurs
    for entry in dataset:
        parts = entry.split(' ')
        if len(parts) == 2:
            tag, summoner_name = parts
            player_info[summoner_name].append(tag)

    # Identifier les joueurs ayant utilisé des tags d'anciennes équipes
    players_to_fix = {}
    for summoner_name, tags in player_info.items():
        if len(tags) > 1:
            players_to_fix[summoner_name] = tags

    # Remplacer les tags d'anciennes équipes par les tags les plus courants
    fixed_dataset = []
    for entry in dataset:
        parts = entry.split(' ')
        if len(parts) == 2:
            tag, summoner_name = parts
            if summoner_name in players_to_fix:
                fixed_tag = max(player_info[summoner_name], key=player_info[summoner_name].count)
                entry = f"{fixed_tag} {summoner_name}"
            fixed_dataset.append(entry)

    return fixed_dataset


final_dataset["summonerName"] = fix_tags(final_dataset["summonerName"])

# Print the players that are present less than 10 times
player_counts = final_dataset["summonerName"].value_counts()
# Since every player must have played 18
print(player_counts[player_counts < 18])



summonerName
IZI Djoko        16
VITB Scarface    10
VITB PatkicaA     8
IZI Manaty        2
Name: count, dtype: int64


# Analysis of the data

In [265]:
# Possibility to make a analysis of the data

# profile = ProfileReport(final_dataset, title="Profiling Report")
# profile.to_notebook_iframe()

# Derived attributes
In his [analysis](https://medium.com/@benjamin.castet/rate-a-single-performance-league-of-legends-and-other-competitive-games-1e45fe7b9fdf) of the stats, Xenesis used some derived attributes. I'll try to explain them here. 

## Champ-Role-Patch
Firstly, we have the 'Champ-Role-Patch' attribute. It is the concatenation of the champion, the role and the patch. It is used to group the stats by champion, role and patch. As Xenesis writed explicitly, this is useful to make a difference between the stats of a champion in a different role. For example, Mid Seraphine and Support Seraphine are not the same champion since Mid Seraphine is played more as a poke champion and Support Seraphine is played more as an enchanter. 

Also we add the patch in this attribute because the build or the champion can change through the patches. For example, Aatrox can be built as a bruiser or as a tank in patch 11.1 and the next patch nerfs the tank build but the champion is still playable with lethality. 


In [266]:
# Make column for Champion-Role-Patch
final_dataset['champ-role-patch'] = final_dataset['championName'] + '-' + final_dataset['role'] + '-' + final_dataset['patch'].astype(str)

## Players stats against whole opponent team
Something good that I saw from @lol_Xenesis on Twitter is to compare the values of every players stats 
against the whole opponent team.
It will basically give you something like :
- playerWardsKilled / wholeOpponentTeamWardsKilled

This is kinda smart because it doesn't require to compare with the main matchup since you can have weak side and strong sides. 
By doing this we will see if each players had a advantage over the opponents.

For example, imagine a composition with a advantage in the matchup on top. It will be normal to see the top laner having a better stats than the opponent. But obviously, the opponent will find a draft with some advantages on the other lanes (with a strong bot lane for example). This is why we need to compare the stats of the top laner with the whole opponent team.

In [267]:
# Let's try to make new features

# If we make a basic (K+A)/D it will tend to inf since it's possible to have division by 0 (deaths)
# final_dataset["kda"] =  (final_dataset["kills"] + final_dataset["assists"]) / final_dataset["deaths"] 
# DON'T DO THIS 


final_dataset["k+a"] = final_dataset["kills"] + final_dataset["assists"]

# ---------------------------
# First rename the columns
# ---------------------------

# Rename the column "totalGold" to "gold"
final_dataset.rename(columns={"goldEarned": "gold"}, inplace=True)

# Rename totalHeal to heal
final_dataset.rename(columns={"totalHeal": "heal"}, inplace=True)

# Rename totalDamageDealtToChampions to damage
final_dataset.rename(columns={"totalDamageDealtToChampions": "damage"}, inplace=True)

# Rename totalDamageTaken to damageTaken
final_dataset.rename(columns={"totalDamageTaken": "damageTaken"}, inplace=True)

# Rename totalMinionsKilled to cs
final_dataset.rename(columns={"totalMinionsKilled": "cs"}, inplace=True)

# Rename timeCCingOthers to cc
final_dataset.rename(columns={"timeCCingOthers": "cc"}, inplace=True)

In [268]:
# Deaths
final_dataset['total'] = final_dataset.groupby(['gameId', 'teamId'])['deaths'].transform('sum')
# If the total is equal to 0, then we set it to 0.01 to avoid division by 0
final_dataset['total'] = final_dataset['total'].apply(lambda x: 0.01 if x == 0 else x)

final_dataset['n_deaths'] = final_dataset.apply(lambda row: row['deaths'] / final_dataset[(final_dataset['gameId'] == row['gameId']) & (final_dataset['teamId'] != row['teamId'])]['total'].iloc[0], axis=1)
contains_infinite = np.isinf(final_dataset['n_deaths']).any()
print(contains_infinite)

False


In [269]:
# k+a
final_dataset['total'] = final_dataset.groupby(['gameId', 'teamId'])['k+a'].transform('sum')
# Add k+a of the player game in total column
# Otherwise it can have infinite values
final_dataset['total'] = final_dataset.apply(lambda row: row['total'] + final_dataset[(final_dataset['gameId'] == row['gameId']) & (final_dataset['teamId'] != row['teamId'])]['k+a'].iloc[0], axis=1)

final_dataset['n_k+a'] = final_dataset.apply(lambda row: row['k+a'] / final_dataset[(final_dataset['gameId'] == row['gameId']) & (final_dataset['teamId'] != row['teamId'])]['total'].iloc[0], axis=1)

contains_infinite = np.isinf(final_dataset['n_k+a']).any()
print(contains_infinite)
#final_dataset[["k+a", "total","n_k+a"]].head(10)

False


In [270]:
# damage
final_dataset['total'] = final_dataset.groupby(['gameId', 'teamId'])['damage'].transform('sum')
final_dataset['n_damage'] = final_dataset.apply(lambda row: row['damage'] / final_dataset[(final_dataset['gameId'] == row['gameId']) & (final_dataset['teamId'] != row['teamId'])]['total'].iloc[0], axis=1)

In [271]:
# gold
final_dataset['total'] = final_dataset.groupby(['gameId', 'teamId'])['gold'].transform('sum')
final_dataset['n_gold'] = final_dataset.apply(lambda row: row['damage'] / final_dataset[(final_dataset['gameId'] == row['gameId']) & (final_dataset['teamId'] != row['teamId'])]['total'].iloc[0], axis=1)

In [272]:
# cs
final_dataset['total'] = final_dataset.groupby(['gameId', 'teamId'])['cs'].transform('sum')
final_dataset['n_cs'] = final_dataset.apply(lambda row: row['cs'] / final_dataset[(final_dataset['gameId'] == row['gameId']) & (final_dataset['teamId'] != row['teamId'])]['total'].iloc[0], axis=1)

In [273]:
# champLevel
final_dataset['total'] = final_dataset.groupby(['gameId', 'teamId'])['champExperience'].transform('sum')
final_dataset['n_champExperience'] = final_dataset.apply(lambda row: row['champExperience'] / final_dataset[(final_dataset['gameId'] == row['gameId']) & (final_dataset['teamId'] != row['teamId'])]['total'].iloc[0], axis=1)


In [274]:
# wardsPlaced
final_dataset['total'] = final_dataset.groupby(['gameId', 'teamId'])['wardsPlaced'].transform('sum')
final_dataset['n_wardsPlaced'] = final_dataset.apply(lambda row: row['wardsPlaced'] / final_dataset[(final_dataset['gameId'] == row['gameId']) & (final_dataset['teamId'] != row['teamId'])]['total'].iloc[0], axis=1)

In [275]:
# wardsKilled
final_dataset['total'] = final_dataset.groupby(['gameId', 'teamId'])['wardsKilled'].transform('sum')
final_dataset['n_wardsKilled'] = final_dataset.apply(lambda row: row['wardsKilled'] / final_dataset[(final_dataset['gameId'] == row['gameId']) & (final_dataset['teamId'] != row['teamId'])]['total'].iloc[0], axis=1)

In [276]:
# heal
final_dataset['total'] = final_dataset.groupby(['gameId', 'teamId'])['heal'].transform('sum')
final_dataset['n_heal'] = final_dataset.apply(lambda row: row['heal'] / final_dataset[(final_dataset['gameId'] == row['gameId']) & (final_dataset['teamId'] != row['teamId'])]['total'].iloc[0], axis=1)

In [277]:
# damageSelfMitigated
final_dataset['total'] = final_dataset.groupby(['gameId', 'teamId'])['damageSelfMitigated'].transform('sum')
final_dataset['n_damageSelfMitigated'] = final_dataset.apply(lambda row: row['damageSelfMitigated'] / final_dataset[(final_dataset['gameId'] == row['gameId']) & (final_dataset['teamId'] != row['teamId'])]['total'].iloc[0], axis=1)

In [278]:
# cc
final_dataset['total'] = final_dataset.groupby(['gameId', 'teamId'])['cc'].transform('sum')
final_dataset['n_cc'] = final_dataset.apply(lambda row: row['cc'] / final_dataset[(final_dataset['gameId'] == row['gameId']) & (final_dataset['teamId'] != row['teamId'])]['total'].iloc[0], axis=1)

In [279]:
# damageTaken
final_dataset['total'] = final_dataset.groupby(['gameId', 'teamId'])['damageTaken'].transform('sum')
final_dataset['n_damageTaken'] = final_dataset.apply(lambda row: row['damageTaken'] / final_dataset[(final_dataset['gameId'] == row['gameId']) & (final_dataset['teamId'] != row['teamId'])]['total'].iloc[0], axis=1)

final_dataset = final_dataset.drop(['total'], axis=1)

In [280]:
# profile = ProfileReport(final_dataset, title="Profiling Report")
# profile.to_file("profiling_report.html")
# # Now let's show the report with the new attributes

# Checks
Here we are checking the number of games for each champion, role and patch

In [281]:

champ_role_patch_counts = final_dataset['champ-role-patch'].value_counts()

print(champ_role_patch_counts)

champ-role-patch
Kaisa-ADC-13.13          21
Aphelios-ADC-13.11       18
Rell-SUPPORT-13.13       17
Xayah-ADC-13.13          16
Aphelios-ADC-13.1        16
                         ..
Gragas-JUNGLE-13.1        1
Volibear-JUNGLE-13.13     1
Camille-TOP-13.1          1
Jayce-TOP-13.1            1
Samira-ADC-13.14          1
Name: count, Length: 254, dtype: int64


# Correlation coefficients
Now we want to get the correlation coefficients for each champ-role-patch possible value. 

As said before, if a champion is played as support in a game, its stats won't be the same as if it was played as mid. Also the goal of the champion will probably not be the same. So we want to get the correlation coefficients for each champ-role-patch possible value.

In [282]:

# Load or define your dataset as 'final_dataset'
# Define the column on which you want to apply the filter
filter_column = 'champ-role-patch'

# Define the numeric columns for which you want to calculate correlations
numeric_columns = final_dataset.filter(regex='^n_').columns.tolist() + ['win']

# Create an empty dictionary to store correlation coefficients for each group
correlation_coefficients = {}

# Group by the filter_column and calculate correlation matrices for each group
for group_name, group_data in final_dataset.groupby(filter_column):
    correlation_matrix = group_data[numeric_columns].corr()
    win_correlation = correlation_matrix['win'].drop('win')  # Get correlation coefficients for 'win'
    # Remove the win coefficient from the correlation matrix

    correlation_coefficients[group_name] = win_correlation

# Print or display the correlation coefficients for each group
for group_name, win_correlation in correlation_coefficients.items():
    print(f"Correlation coefficients for 'win' {group_name}:\n{win_correlation}")
    print("\n")


Correlation coefficients for 'win' Aatrox-TOP-13.1:
n_deaths                NaN
n_k+a                   NaN
n_damage                NaN
n_gold                  NaN
n_cs                    NaN
n_champExperience       NaN
n_wardsPlaced           NaN
n_wardsKilled           NaN
n_heal                  NaN
n_damageSelfMitigated   NaN
n_cc                    NaN
n_damageTaken           NaN
Name: win, dtype: float64


Correlation coefficients for 'win' Aatrox-TOP-13.11:
n_deaths                NaN
n_k+a                   NaN
n_damage                NaN
n_gold                  NaN
n_cs                    NaN
n_champExperience       NaN
n_wardsPlaced           NaN
n_wardsKilled           NaN
n_heal                  NaN
n_damageSelfMitigated   NaN
n_cc                    NaN
n_damageTaken           NaN
Name: win, dtype: float64


Correlation coefficients for 'win' Aatrox-TOP-13.14:
n_deaths                -0.584254
n_k+a                   -0.362604
n_damage                -0.396672
n_gold      

# Standardization
We need to standardize the data before making any calculations. By doing this, we ensure that the data is comparable and all in the same scale (without coefficients).
 

In [283]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Load or define your dataset as 'final_dataset'
# For example:
# final_dataset = pd.read_csv('your_dataset.csv')

# Select numeric columns that start with 'n_'
numeric_columns = final_dataset.filter(regex='^n_').columns.tolist()

# Create a StandardScaler instance
scaler = StandardScaler()

# Apply standardization to the selected numeric columns
final_dataset[numeric_columns] = scaler.fit_transform(final_dataset[numeric_columns])

# Now the columns starting with 'n_' are standardized

In [284]:
import pandas as pd

# Load or define your dataset as 'final_dataset'
# Define the column on which you want to apply the filter
filter_column = 'champ-role-patch'

# Create a copy of the dataset to store the ratings
ratings_dataset = final_dataset.copy()

# Apply ratings to the numeric columns based on correlation coefficients
for group_name, win_correlation in correlation_coefficients.items():
    group_indices = final_dataset[filter_column] == group_name
    ratings = final_dataset.loc[group_indices, numeric_columns] * win_correlation.values
    ratings_dataset.loc[group_indices, 'rating'] = ratings.sum(axis=1)



# Ratings
So basically, we have 2 types of ratings:
- **Positive** ratings 
- **Negative** ratings

When the rating is positive it means that the player is better than the average player of his role, champion and patch
When the rating is negative it means that the player is worse than the average player of his role, champion and patch

In [285]:
ratings_dataset.tail(10)

,gameId,patch,role,summonerName,blueTeam,redTeam,teamId,championName,kills,deaths,...,n_gold,n_cs,n_champExperience,n_wardsPlaced,n_wardsKilled,n_heal,n_damageSelfMitigated,n_cc,n_damageTaken,rating
900,2691384,13.14,TOP,BKR Howling,BK ROG Esports,Team GO,100,Gnar,0,1,...,-0.505591,0.595089,1.484288,-0.570001,-0.791001,0.107396,0.149244,0.435671,0.499191,0.000000
901,2691384,13.14,JUNGLE,BKR Pridestalkr,BK ROG Esports,Team GO,100,Vi,2,2,...,-0.477578,-1.240066,0.258226,-0.758514,0.568809,1.251711,0.613729,0.014904,0.268963,0.000000
902,2691384,13.14,MID,BKR Czekolad,BK ROG Esports,Team GO,100,Azir,10,2,...,1.260627,0.941061,1.525349,-0.522873,-1.244271,-0.770654,-0.756675,-0.405863,-0.881418,0.000000
903,2691384,13.14,ADC,BKR Trigger,BK ROG Esports,Team GO,100,Kaisa,3,2,...,-0.211994,0.820723,0.881980,0.419692,1.626438,-0.517122,-0.950082,-1.089609,-1.182291,3.780587
904,2691384,13.14,SUPPORT,BKR Veignorem,BK ROG Esports,Team GO,100,Nautilus,2,5,...,-1.219491,-1.142291,-0.900362,2.022053,0.568809,-0.663653,-0.457215,0.961630,-0.741515,-0.389774
905,2691384,13.14,TOP,GO Ragner,BK ROG Esports,Team GO,200,Malphite,0,1,...,-0.392533,0.672523,0.418617,-0.379511,-0.761278,-0.287601,1.751611,0.267571,0.102812,0.000000
906,2691384,13.14,JUNGLE,GO Lyncas,BK ROG Esports,Team GO,200,Sejuani,0,5,...,-0.983300,-1.232345,-1.257465,-0.719625,0.174984,1.203419,0.587846,-0.023253,1.818567,-0.214438
907,2691384,13.14,MID,GO toucouille,BK ROG Esports,Team GO,200,Jayce,1,3,...,-0.346853,0.792989,-0.341222,-0.592082,-0.627527,-0.810099,-0.790591,-0.798785,-0.218307,-1.242361
908,2691384,13.14,ADC,GO Jezu,BK ROG Esports,Team GO,200,Samira,9,3,...,0.003859,0.943571,-0.217767,-0.634597,0.442488,0.472001,-0.336963,-0.847255,0.332183,0.000000
909,2691384,13.14,SUPPORT,GO kamilius,BK ROG Esports,Team GO,200,Rell,2,5,...,-1.339793,-1.187170,-1.880669,1.661176,-0.493775,-0.825866,-0.174222,0.509924,0.056334,-3.534051


In [286]:
# Now we will make an average of the ratings for each summonerName

# Group by 'summonerName' and 'champ-role-patch', calculate the average rating and count for each group
grouped_data = ratings_dataset.groupby(['summonerName', 'role'])['rating'].agg(['mean', 'count'])

# Create a new DataFrame to store the average ratings, counts, summoner names, and champ-role-patch
average_ratings_df = pd.DataFrame({
    'summonerName': grouped_data.index.get_level_values('summonerName'),
    'role': grouped_data.index.get_level_values('role'),
    'average_rating': grouped_data['mean'],
    'count': grouped_data['count']
}).reset_index(drop=True)  # Reset index and drop the old index levels

sorted_average_ratings_df = average_ratings_df.sort_values(by='average_rating', ascending=False)
print(sorted_average_ratings_df.head(10))

       summonerName    role  average_rating  count
32       KC Caliste     ADC        3.019087     18
12  BKR Pridestalkr  JUNGLE        2.220873     19
47      VITB Daglas  JUNGLE        2.183217     18
34         KC SAKEN     MID        2.015386     18
13      BKR Trigger     ADC        1.962067     19
15          GO Jezu     ADC        1.875642     19
10     BKR Czekolad     MID        1.828327     19
43      SLY Shlatan  JUNGLE        1.721043     18
33       KC Cinkrof  JUNGLE        1.678313     18
31     KC Cabochard     TOP        1.567775     18


In [287]:
# Group by 'role' and display the top 5 players for each role
for role, group_data in sorted_average_ratings_df.groupby('role'):
    top_players = group_data.head(5)
    print(f"Top 5 players for role '{role}':\n{top_players}\n")

Top 5 players for role 'ADC':
   summonerName role  average_rating  count
32   KC Caliste  ADC        3.019087     18
13  BKR Trigger  ADC        1.962067     19
15      GO Jezu  ADC        1.875642     19
49    VITB Neon  ADC        1.316414     18
45  SLY TakeSet  ADC        1.191061     18

Top 5 players for role 'JUNGLE':
       summonerName    role  average_rating  count
12  BKR Pridestalkr  JUNGLE        2.220873     19
47      VITB Daglas  JUNGLE        2.183217     18
43      SLY Shlatan  JUNGLE        1.721043     18
33       KC Cinkrof  JUNGLE        1.678313     18
4      AEG SkewMond  JUNGLE        1.354908     18

Top 5 players for role 'MID':
    summonerName role  average_rating  count
34      KC SAKEN  MID        2.015386     18
10  BKR Czekolad  MID        1.828327     19
42      SLY Peng  MID        1.357404     18
46   VITB Czajek  MID        1.294367     18
8    BDSA Reeker  MID        1.137122     18

Top 5 players for role 'SUPPORT':
     summonerName     role  av

In [288]:
# Define the role you want to display
target_role = 'TOP'  # Replace with the desired role

# Filter the DataFrame for the target role and display the top 5 players
top_players_target_role = sorted_average_ratings_df[sorted_average_ratings_df['role'] == target_role]
print(f"Players for role '{target_role}':\n{top_players_target_role}")

Players for role 'JUNGLE':
       summonerName    role  average_rating  count
12  BKR Pridestalkr  JUNGLE        2.220873     19
47      VITB Daglas  JUNGLE        2.183217     18
43      SLY Shlatan  JUNGLE        1.721043     18
33       KC Cinkrof  JUNGLE        1.678313     18
4      AEG SkewMond  JUNGLE        1.354908     18
16        GO Lyncas  JUNGLE        1.319384     19
25        IZI Djoko  JUNGLE        0.324109     16
7         BDSA Maxi  JUNGLE        0.226590     18
21         GW Eckas  JUNGLE        0.058225     18
39       LDLC White  JUNGLE       -0.659887     18
28       IZI Manaty  JUNGLE       -1.922865      2


In [289]:
# See all the ratings of the summonerName 'KC Caliste'

ratings_dataset[ratings_dataset['summonerName'] == 'KC Caliste']


,gameId,patch,role,summonerName,blueTeam,redTeam,teamId,championName,kills,deaths,...,n_gold,n_cs,n_champExperience,n_wardsPlaced,n_wardsKilled,n_heal,n_damageSelfMitigated,n_cc,n_damageTaken,rating
28,2815467,13.10,ADC,KC Caliste,Team BDS Academy,Karmine Corp,200,Aphelios,6,0,...,2.589587,1.264922,1.071174,-0.294482,-0.009501,0.084040,-0.272462,0.274256,-1.224893,9.001533
88,2815530,13.10,ADC,KC Caliste,Solary,Karmine Corp,200,Zeri,7,1,...,0.965374,1.069761,1.113125,-0.561047,2.230798,-0.346548,-0.355838,-0.888633,-0.460931,0.013773
138,2817850,13.10,ADC,KC Caliste,Aegis (French Team),Karmine Corp,200,Aphelios,2,4,...,2.375743,1.180416,-0.015190,-0.488389,0.007295,-0.357881,-0.290646,-0.603313,-0.597021,2.261102
173,2817900,13.10,ADC,KC Caliste,Karmine Corp,LDLC OL,100,Lucian,9,1,...,1.713485,0.864880,0.824222,-0.230420,1.158060,-0.925989,-0.533296,-1.089609,-0.583628,7.689322
233,2685225,13.11,ADC,KC Caliste,Karmine Corp,Vitality.Bee,100,Jinx,1,4,...,-0.378626,0.815412,-0.363722,-0.469854,0.499075,-0.416332,-0.873630,0.026781,0.226365,-2.178964
273,2685282,13.11,ADC,KC Caliste,Karmine Corp,GameWard,100,Lucian,3,2,...,0.732038,0.803859,0.377852,-0.477091,1.701983,-0.458144,-0.817401,-1.013547,-0.225529,2.126843
323,2684838,13.11,ADC,KC Caliste,Karmine Corp,Izi Dream,100,Lucian,7,3,...,2.548488,1.331835,0.961985,-0.338130,1.640174,-0.530797,-0.921823,-0.927510,-0.734191,6.955023
383,2683954,13.11,ADC,KC Caliste,Karmine Corp,Team GO,100,Aphelios,1,4,...,0.733780,0.740285,-0.698691,-0.476579,0.603676,-0.757325,-0.548944,-0.532537,-0.512668,-0.802875
438,2686330,13.11,ADC,KC Caliste,BK ROG Esports,Karmine Corp,200,Aphelios,12,5,...,3.015558,1.148518,0.559707,-0.354342,0.929888,-0.677287,-0.705796,-0.326818,-1.437460,2.853210
478,2687403,13.11,ADC,KC Caliste,Team BDS Academy,Karmine Corp,200,Aphelios,8,2,...,3.145641,1.788997,1.117074,-0.487257,3.675366,-0.246412,-0.655229,-0.477726,-1.118125,4.391005
